## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
app_cat = application_df.dtypes.index.tolist()
application_df[app_cat].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df['APPLICATION_TYPE'].value_counts()
app_type_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_count[app_type_count < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count.head(20)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: count, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_app_df = pd.get_dummies(application_df)
numeric_app_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [11]:
# Split our preprocessed data into our features and target arrays
y = numeric_app_df["IS_SUCCESSFUL"]
X = numeric_app_df.drop(["IS_SUCCESSFUL"],axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 396       
                                                                 
 dense_1 (Dense)             (None, 18)                180       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 595 (2.32 KB)
Trainable params: 595 (2.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5419 - accuracy: 0.7374
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7376
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 0.7369
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5418 - accuracy: 0.7369
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7368
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7371
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7374
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7369
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5410 - accuracy: 0.7372
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5523 - accuracy: 0.7263 - 512ms/epoch - 2ms/step
Loss: 0.5522637963294983, Accuracy: 0.7262973785400391


In [18]:
# Export our model to HDF5 file
nn.save('Models/AlphabetSoupCharity1.h5')

In [21]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [23]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=300)


Epoch 1/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7379
Epoch 2/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5382 - accuracy: 0.7388
Epoch 3/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7393
Epoch 4/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7393
Epoch 5/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7387
Epoch 6/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7393
Epoch 7/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7382
Epoch 8/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5379 - accuracy: 0.7384
Epoch 9/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7382
Epoch 10/300
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accura

In [24]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5526 - accuracy: 0.7272 - 355ms/epoch - 1ms/step
Loss: 0.552565336227417, Accuracy: 0.7272303104400635


In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 18
hidden_nodes_layer3 = 36
hidden_nodes_layer4 = 72
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 9)                 396       
                                                                 
 dense_28 (Dense)            (None, 18)                180       
                                                                 
 dense_29 (Dense)            (None, 36)                684       
                                                                 
 dense_30 (Dense)            (None, 72)                2664      
                                                                 
 dense_31 (Dense)            (None, 1)                 73        
                                                                 
Total params: 3997 (15.61 KB)
Trainable params: 3997 (15.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [45]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=500)


Epoch 1/500
804/804 [==============================] - 2s 1ms/step - loss: 0.5954 - accuracy: 0.6968
Epoch 2/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5617 - accuracy: 0.7283
Epoch 3/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5568 - accuracy: 0.7305
Epoch 4/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5557 - accuracy: 0.7305
Epoch 5/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7316
Epoch 6/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5533 - accuracy: 0.7309
Epoch 7/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7321
Epoch 8/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7333
Epoch 9/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7327
Epoch 10/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accura

In [46]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5595 - accuracy: 0.7242 - 477ms/epoch - 2ms/step
Loss: 0.5595447421073914, Accuracy: 0.7241982221603394


In [47]:
nn.save('Models/AlphabetSoupCharity2.h5')

In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 35
hidden_nodes_layer3= 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 20)                880       
                                                                 
 dense_37 (Dense)            (None, 35)                735       
                                                                 
 dense_38 (Dense)            (None, 50)                1800      
                                                                 
 dense_39 (Dense)            (None, 1)                 51        
                                                                 
Total params: 3466 (13.54 KB)
Trainable params: 3466 (13.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
fit_nn= nn.fit(X_train_scaled, y_train, epochs=1000)


Epoch 1/1000
804/804 [==============================] - 2s 1ms/step - loss: 0.5290 - accuracy: 0.7434
Epoch 2/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5291 - accuracy: 0.7435
Epoch 3/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5288 - accuracy: 0.7432
Epoch 4/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5286 - accuracy: 0.7428
Epoch 5/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5286 - accuracy: 0.7428
Epoch 6/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5290 - accuracy: 0.7430
Epoch 7/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5286 - accuracy: 0.7429
Epoch 8/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5287 - accuracy: 0.7433
Epoch 9/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.5289 - accuracy: 0.7429
Epoch 10/1000
804/804 [==============================] - 1s 1ms/step - loss: 0.528

In [78]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5834 - accuracy: 0.7269 - 424ms/epoch - 2ms/step
Loss: 0.5833809971809387, Accuracy: 0.7268804907798767


In [79]:
nn.save('Models/AlphabetSoupCharity3.h5')